In [3]:
import h2o
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
; Java HotSpot(TM) 64-Bit Server VM (build 23.0.1+11-39, mixed mode, sharing)
  Starting server from C:\Users\alonm\AppData\Local\Programs\Python\Python312\Lib\site-packages\h2o\backend\bin\h2o.jar
  Ice root: C:\Users\alonm\AppData\Local\Temp\tmpd3fp3_o4
  JVM stdout: C:\Users\alonm\AppData\Local\Temp\tmpd3fp3_o4\h2o_alonm_started_from_python.out
  JVM stderr: C:\Users\alonm\AppData\Local\Temp\tmpd3fp3_o4\h2o_alonm_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,02 secs
H2O_cluster_timezone:,America/Chicago
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.46.0.5
H2O_cluster_version_age:,2 months and 27 days
H2O_cluster_name:,H2O_from_python_alonm_adpqca
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.423 Gb
H2O_cluster_total_cores:,16
H2O_cluster_allowed_cores:,16
H2O_cluster_status:,"locked, healthy"


In [4]:
import os

input_path = 'presidential_turnout_autoML'
file_names = os.listdir(input_path)
for i, file_name in enumerate(file_names):
    model_path = os.path.join(input_path, file_name)
    model = h2o.load_model(model_path)
    print(f'Model {i} loaded.')
    model_summary = model.summary()
    print(model_summary)

Model 0 loaded.
Status of Neuron Layers: predicting presidential_votes_ratio, regression, gaussian distribution, Quadratic loss, 501 weights/biases, 10,5 KB, 360.360 training samples, mini-batch size 1
    layer    units    type              dropout    l1    l2    mean_rate             rate_rms              momentum    mean_weight           weight_rms          mean_bias            bias_rms
--  -------  -------  ----------------  ---------  ----  ----  --------------------  --------------------  ----------  --------------------  ------------------  -------------------  -----------------------
    1        8        Input             0.0
    2        50       RectifierDropout  40.0       0.0   0.0   0.014593527158140204  0.029078200459480286  0.0         -0.03609370522317477  0.489656925201416   -0.8225816972471045  0.2532920837402344
    3        1        Linear                       0.0   0.0   0.009569003623910248  0.01558372750878334   0.0         0.3735393318533897    0.4319138526916